In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch

In [2]:
inputs = pd.read_csv("heart.csv")
targets = inputs.pop("target")

inputs = np.array(inputs)
targets = np.array(targets)

print(inputs)
print(targets)


[[63.  1.  3. ...  0.  0.  1.]
 [37.  1.  2. ...  0.  0.  2.]
 [41.  0.  1. ...  2.  0.  2.]
 ...
 [68.  1.  0. ...  1.  2.  3.]
 [57.  1.  0. ...  1.  1.  3.]
 [57.  0.  1. ...  1.  1.  2.]]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]


In [3]:
inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs, targets, test_size=0.33)

print(inputs_train.shape, targets_train.shape)
print(inputs_test.shape, targets_test.shape)
targets_train

(203, 13) (203,)
(100, 13) (100,)


array([1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 0], dtype=int64)

In [4]:
inputs_train = StandardScaler().fit_transform(inputs_train)
inputs_test = StandardScaler().fit_transform(inputs_test)

In [5]:
inputs_train = torch.from_numpy(inputs_train.astype(np.float32))
inputs_test = torch.from_numpy(inputs_test.astype(np.float32))
targets_train = torch.from_numpy(targets_train.astype(np.float32))
targets_test = torch.from_numpy(targets_test.astype(np.float32))


In [6]:
targets_train = targets_train.reshape(-1,1)
targets_test = targets_test.reshape(-1,1)
targets_test.shape

torch.Size([100, 1])

In [7]:
class HeartModel(torch.nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.linear = torch.nn.Linear(input_dim,output_dim)
    
    def forward(self, inputs):
        targets_pred = torch.sigmoid(self.linear(inputs))
        return targets_pred

In [8]:
#model
logistic_model = HeartModel(13,1)


In [9]:
#loss function
loss_fn = torch.nn.BCELoss()

In [10]:
#optimizer
opt = torch.optim.SGD(logistic_model.parameters(), lr = 1e-2)

In [11]:
def fit(num_epoch, model, loss_function, optimizer, inputs_train,inputs_test,targets_train, targets_test):
    for epoch in range(num_epoch):
        targets_pred = model(inputs_train)
        
        loss = loss_function(targets_pred, targets_train)
        
        loss.backward()
        
        optimizer.step()
        optimizer.zero_grad()
        
        if (epoch+1)%10 == 0:
            print("Epoch: ", epoch+1, "  Loss: ", loss.item())
    
            with torch.no_grad():
                test_targets_pred = model(inputs_test)
                test_targets_pred_class = test_targets_pred.round()
                
                acc = test_targets_pred_class.eq(targets_test).sum()/len(targets_test)
                print("Accuracy:" , int(100*acc.item()),"%")

In [12]:
fit(300, logistic_model, loss_fn, opt, inputs_train, inputs_test, targets_train, targets_test)

Epoch:  10   Loss:  0.6200892329216003
Accuracy: 74 %
Epoch:  20   Loss:  0.5991184711456299
Accuracy: 74 %
Epoch:  30   Loss:  0.5804542899131775
Accuracy: 75 %
Epoch:  40   Loss:  0.5637960433959961
Accuracy: 76 %
Epoch:  50   Loss:  0.5488828420639038
Accuracy: 76 %
Epoch:  60   Loss:  0.5354897379875183
Accuracy: 79 %
Epoch:  70   Loss:  0.5234233736991882
Accuracy: 80 %
Epoch:  80   Loss:  0.5125175714492798
Accuracy: 80 %
Epoch:  90   Loss:  0.5026298761367798
Accuracy: 81 %
Epoch:  100   Loss:  0.4936375021934509
Accuracy: 82 %
Epoch:  110   Loss:  0.4854351878166199
Accuracy: 81 %
Epoch:  120   Loss:  0.47793200612068176
Accuracy: 81 %
Epoch:  130   Loss:  0.47104933857917786
Accuracy: 81 %
Epoch:  140   Loss:  0.4647192060947418
Accuracy: 81 %
Epoch:  150   Loss:  0.4588824510574341
Accuracy: 81 %
Epoch:  160   Loss:  0.45348766446113586
Accuracy: 81 %
Epoch:  170   Loss:  0.44848981499671936
Accuracy: 81 %
Epoch:  180   Loss:  0.44384947419166565
Accuracy: 81 %
Epoch:  190   